<a href="https://colab.research.google.com/github/luiscunhacsc/finance_python/blob/main/24_Risk_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Risk Monitoring and Reporting

**Continuous Monitoring:** Regularly review and update risk assessments.

**Key Metrics:** Track VaR, CVaR, beta, and other risk indicators.

**Reporting:** Communicate risk levels and mitigation strategies to stakeholders.

**Tools:** Use risk management software and dashboards for real-time monitoring.


## Practical Example

**Scenario:** Create a risk dashboard to monitor key risk metrics.

**Steps:**
1. Load historical stock price data.
2. Calculate key risk metrics (VaR, CVaR, beta).
3. Visualize the metrics using a dashboard layout.


## Installing Required Libraries

Before running the code, make sure to install the necessary libraries. We need `dash` for creating the dashboard and `yfinance` to download the historical stock data from Yahoo Finance.


In [ ]:
%pip install -q dash
%pip install -q yfinance


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Downloading Historical Stock Data

We'll use the `yfinance` library to download the historical stock data for specific tickers and save it as a CSV file.


In [ ]:
import yfinance as yf
import pandas as pd

# List of stock tickers
tickers = ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']

# Download historical data for selected tickers
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Adj Close']

# Save the data to a CSV file
data.to_csv('historical_stock_prices.csv')

# Display the first few rows of the data
data.head()


[*********************100%%**********************]  5 of 5 completed


Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,72.876106,94.900497,68.355545,153.938187,28.684000
2020-01-03,72.167625,93.748497,67.997955,152.021393,29.534000
2020-01-06,72.742661,95.143997,69.810379,152.414383,30.102667
2020-01-07,72.400551,95.343002,69.675537,151.024658,31.270666
2020-01-08,73.565193,94.598503,70.171463,153.430252,32.809334


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd

# Load dataset
data = pd.read_csv('historical_stock_prices.csv', index_col='Date', parse_dates=True)
prices = data['AAPL']  # Example: using AAPL data

# Calculate risk metrics
returns = prices.pct_change().dropna()
VaR_95 = np.percentile(returns, 5)
CVaR_95 = returns[returns < VaR_95].mean()
covariance = np.cov(returns, returns)[0][1]
market_variance = np.var(returns)
beta = covariance / market_variance

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Risk Dashboard"),
    dcc.Graph(
        id='returns-histogram',
        figure={
            'data': [
                {'x': returns, 'type': 'histogram', 'name': 'Returns'},
            ],
            'layout': {
                'title': 'Distribution of Stock Returns',
                'shapes': [
                    {
                        'type': 'line', 'x0': VaR_95, 'x1': VaR_95, 'y0': 0, 'y1': 1,
                        'xref': 'x', 'yref': 'paper',
                        'line': {'color': 'red', 'width': 2, 'dash': 'dash'}
                    }
                ]
            }
        }
    ),
    html.Div([
        html.H3(f"VaR (95%): {VaR_95:.4f}"),
        html.H3(f"CVaR (95%): {CVaR_95:.4f}"),
        html.H3(f"Beta: {beta:.4f}"),
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\luisl\AppData\Local\Temp/ipykernel_33904/3272841324.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\luisl\AppData\Local\Temp/ipykernel_33904/3272841324.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Risk Dashboard Output

The dashboard provides key risk metrics for the selected stock portfolio:

- **VaR (95%)**: The Value at Risk (VaR) at the 95% confidence level is -0.0348. This means that there is a 5% chance that the portfolio could lose at least 3.48% of its value in a single day based on historical data.
  
- **CVaR (95%)**: The Conditional Value at Risk (CVaR) at the 95% confidence level is -0.0509. CVaR estimates the expected loss assuming that the loss exceeds the VaR threshold. This implies that, on average, the loss could be 5.09% on the worst 5% of days.
  
- **Beta**: The Beta value of 1.0013 indicates that the portfolio has a volatility similar to the overall market (S&P 500), meaning the portfolio's movements are expected to be in line with the market.

The histogram in the dashboard visualizes the distribution of daily returns for the portfolio, with the red dashed line indicating the VaR threshold at the 95% confidence level.
